In [1]:
import os

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data # 获取迭代数据
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.autograd import Variable # 获取变量



import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import cooler
import cooltools


In [8]:
# cool_path='raw_data/K562/cooler/GSM2109985_90_K562_SetB-40kb.cool'
# clr = cooler.Cooler(cool_path)

#通过cooler读入GSM2109985_90_K562_SetB-40kb.mcool
clr = cooler.Cooler("raw_data/K562/cooler/GSM2109974_79_K562_SetB.mcool::/resolutions/120000")

In [19]:
chromstarts = []
for i in clr.chromnames:
    # print(f'{i} : {clr.extent(i)}')
    chromstarts.append(clr.extent(i)[0])

In [18]:
# f, ax = plt.subplots(
#     figsize=(7,6))
# im = ax.matshow((clr.matrix(balance=False).fetch('chr1')[:]),vmax=5);
# plt.colorbar(im ,fraction=0.046, pad=0.04, label='raw counts')
# ax.set(xticks=chromstarts, xticklabels=clr.chromnames,
#        xlabel='position, chrom#', ylabel='position, bin#')
# ax.xaxis.set_label_position('top')

## pos data

In [20]:
sv_list = pd.read_csv("./ref_data/sv_list.csv")
k562_list = sv_list[sv_list['Cell Line']=="K562"]

In [71]:
#列出raw_data/K562/cooler/文件夹下所有的文件
file_list = os.listdir("./raw_data/K562/cooler/")
#读取所有的mcool文件
mcool_list = [i for i in file_list if i.endswith(".mcool")]
#读取所有文件名中不包含bulk的
sc_mcool_list = [i for i in mcool_list if "bulk" not in i] #total 34

In [86]:
def get_pos_submatrix(clr,window,label_list):
    bin_table = clr.bins()[:]
    #找到有label对应的位置
    pos_data_list = []
    pos_label_list = []
    
    for i in range(len(label_list)):
        chrom1 = label_list['chrom1'].iat[i]
        chrom2 = label_list['chrom2'].iat[i]
        breakpoint1 = label_list['breakpoint1'].iat[i]
        breakpoint2 = label_list['breakpoint2'].iat[i]
        string = label_list['strands'].iat[i]

        #找到对应的bin
        bin1 = bin_table[(bin_table['chrom']==chrom1) &(bin_table['start'] < breakpoint1 ) & (breakpoint1 < bin_table['end'] )]
        bin2 = bin_table[(bin_table['chrom']==chrom2) &(bin_table['start'] < breakpoint2 ) & (breakpoint2 < bin_table['end'] )]

        #找到对应的matrix
        x1 = bin1.index[0]#得到的是索引
        y1 = bin2.index[0]

        x2 = x1 + window
        y2 = y1 + window
        submatrix = clr.matrix(balance=False)[x1:x2, y1:y2]
        #pos_data_list中添加submatrix
        pos_data_list.append(submatrix)
        pos_label_list.append(string)

    return pos_data_list,pos_label_list



In [104]:
resolution = 120000
window = 28
cool_dir = "raw_data/K562/cooler/"
#读取所有的mcool文件
pos_data_list = []
pos_label_list = []
for mc in sc_mcool_list:
    clr = cooler.Cooler(cool_dir+mc+"::/resolutions/"+str(resolution))
    sc_pos_data_list,sc_pos_label_list = get_pos_submatrix(clr,window,k562_list) 
    pos_data_list.append(sc_pos_data_list)
    pos_label_list.append(sc_pos_label_list)
pos_data_list = [item for sublist in pos_data_list for item in sublist]
pos_label_list = [item for sublist in pos_label_list for item in sublist]
pos_data_list = np.array(pos_data_list)
pos_label_list = np.array(pos_label_list)
#数据类型转换
pos_data_list = pos_data_list.astype(np.float32)
#保存
np.save("input_data/pos_data_list.npy",pos_data_list)
np.save("input_data/pos_label_list.npy",pos_label_list)


## neg data

In [ ]:
def determine_label(tmp_bin_x,tmp_bin_y,k562_list):
    
    label = "no"
    for i in range(len(tmp_bin_x)):
        chr1 = tmp_bin_x['chrom'].iat[i]
        start1 = tmp_bin_x['start'].iat[i]
        end1 = tmp_bin_x['end'].iat[i]
        for j in range(len(tmp_bin_y)):
            chr2 = tmp_bin_y['chrom'].iat[j]
            start2 = tmp_bin_y['start'].iat[j]
            end2 = tmp_bin_y['end'].iat[j]

            for m in range(len(k562_list)):
                chrom1 = k562_list['chrom1'].iat[m]
                chrom2 = k562_list['chrom2'].iat[m]
                breakpoint1 = k562_list['breakpoint1'].iat[m]
                breakpoint2 = k562_list['breakpoint2'].iat[m]
                string = k562_list['strands'].iat[m]
                if (chr1 == chrom1 and chr2==chrom2 and (breakpoint1 > start1 and breakpoint1<end1) and (breakpoint2 > start2 and breakpoint2<end2) ):
                    label = string
                    break
                elif (chr1 == chrom2 and chr2==chrom1 and (breakpoint1 > start2 and breakpoint1<end2) and (breakpoint2 > start1 and breakpoint2<end1)):
                    label = string
                    break
            if label != "no":
                break
        if label != "no":
            break
    return label

          

In [126]:
#很少，但吧
neg_data_list = []
neg_label_list = []


sample_x1=np.random.randint(7000, size=(50))
sample_y1=np.random.randint(7000, size=(50))
sample_x2 = sample_x1 + window
sample_y2 = sample_y1 + window

for x,y in zip(sample_x1,sample_y1):
    submatrix = matrix1[x:x+window, y:y+window]
    nozeron = np.count_nonzero(submatrix)
    tmp_bin_x = bin_table.loc[x:x+window,:]
    tmp_bin_y = bin_table.loc[y:y+window,:]
    #得到标签
    label = determine_label(tmp_bin_x,tmp_bin_y,k562_list)
    if(nozeron>5):
        neg_data_list.append(submatrix)
        neg_label_list.append(label)


In [136]:
all_data = pos_data_list + neg_data_list
all_label = pos_label_list + neg_label_list

In [13]:
# 保存
# a=np.array(all_data)
np.save('data/data.npy',a)   # 保存为.npy格式
# b = np.array(all_label)

# 读取


In [3]:
a = np.load('data/data.npy')

In [16]:
a = a.reshape(47,1,21,21)

In [12]:
#a转成float
a = a.astype(np.float32)

In [189]:
np.unique(b)

array(['++', '+-', '-+', '--', 'no'], dtype='<U2')

In [191]:
label_dic = {'++':0,'+-':1,'-+':2,'--':3,'no':4}

In [192]:
#将b按照label_dic转成对应int
b = [label_dic[i] for i in b]

In [194]:
np.save('data/label.npy',b)

In [149]:
a_tensor = torch.tensor(a,dtype=float)

In [153]:
a_tensor = a_tensor.reshape(47,1,21,21)

In [160]:
#save a_tensor

torch.save(a_tensor, "data/data.pt")





In [161]:
y = torch.load("data/data.pt")

In [200]:

label_file = 'data/label.npy'
data_file = 'data/data.pt' #tensor
